<a href="https://colab.research.google.com/github/simulate111/Basics-of-Programming---Exercise/blob/main/exercise_cross_lingual_transfer_qa_Ex14E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question Answering with zero-shot cross-lingual transfer

In [1]:
!pip install -q transformers sentencepiece datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00


Without sentencepiece installed, the error message is:


```
ValueError: Couldn't instantiate the backend tokenizer from one of:
  (1) a `tokenizers` library serialization file,
  (2) a slow tokenizer instance to convert or
  (3) an equivalent slow tokenizer class to instantiate and convert.

You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

```
[SentencePiece](https://github.com/google/sentencepiece) is an another library for subword tokenization, ie. for splitting words into subwords.


## Create a pipeline for QA

In [2]:
from transformers import pipeline


# a) Get predictions
pipe = pipeline('question-answering', model="deepset/xlm-roberta-large-squad2")
example_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = pipe(example_input, handle_impossible_answer=True)
print(res)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.30940189957618713, 'start': 58, 'end': 133, 'answer': ' gives freedom to the user and let people easily switch between frameworks.'}


## Evaluate on one Finnish example

(English translation also provided)

In [3]:
finnish_input = {
    "question": "Minkä takia linnut pysyvät ilmassa?",
    "context": """Linnut (Aves) ovat tasalämpöisiä, munivia ja höyhenpeitteisiä selkärankaisia,
    joiden siivet antavat useimmille niistä lentokyvyn. Lintuja tunnetaan yli 10 900 lajia.
    Linnut kehittyivät jurakaudella dinosauruksista, ja ne luokitellaankin teropodien alaryhmäksi.
    Lintulajit muistuttavat paljon toisiaan, ja kaikkien lajien perusrakenne on sama. Linnut ovatkin
    muista eläinryhmistä suhteellisen selvästi erottuva ryhmä. Linnuilla on nokka, mutta ei hampaita.
    Eturaajat ovat kehittyneet siiviksi, ja luut ovat onttoja ja kevyitä."""
}
english_input = {
    "question": "Why do birds stay up in the air?",
    "context": """Birds (Aves) are even-tempered, egg-laying and feathered vertebrates,
    whose wings give most of them the ability to fly. There are more than 10 900 known species of birds.
    Birds evolved from dinosaurs in the Jurassic period and are classified as a sub-group of teropods.
    Bird species are very similar to each other, and the basic structure of all species is the same.
    Birds are therefore a relatively distinct group from other animal groups. Birds have beaks but no teeth.
    The forelimbs have evolved into wings and the bones are hollow and light."""
}
print(pipe(finnish_input, handle_impossible_answer=True))
print(pipe(english_input, handle_impossible_answer=True))
print(pipe(english_input))



{'score': 0.5494551658630371, 'start': 88, 'end': 95, 'answer': ' siivet'}
{'score': 0.36224207282066345, 'start': 0, 'end': 0, 'answer': ''}
{'score': 0.07016465812921524, 'start': 79, 'end': 85, 'answer': ' wings'}


## Evaluate using Finnish SQuAD

* DeepL -based machine translation of the English SQuAD v2.0

In [4]:
import datasets
squad_fi = datasets.load_dataset("TurkuNLP/squad_v2_fi", split="validation")

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

squad_v2_fi.py:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

The repository for TurkuNLP/squad_v2_fi contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/TurkuNLP/squad_v2_fi.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] n


ValueError: Loading TurkuNLP/squad_v2_fi requires you to execute the dataset script in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [ ]:
print(squad_fi)
sample = [{"question": e["question"], "context": e["context"]} for e in squad_fi][:1000]
print(len(sample))
predictions = pipe(sample, handle_impossible_answer=True)

In [ ]:
print(predictions[0])
for i in range(10):
  print("Q:", sample[i]["question"])
  print("Pred A:", predictions[i]["answer"])
  print("Corr A:", squad_fi[i]["answers"]["text"])
  print()

In [ ]:
# Use the official SQuAD metric https://huggingface.co/spaces/evaluate-metric/squad_v2
from evaluate import load

squad_metric = load("squad_v2")

gold = []
preds = []

# Reformat the data for evaluation metric
for i in range(len(predictions)):
  g = {'answers': {'answer_start': squad_fi[i]["answers"]["answer_start"], 'text': squad_fi[i]["answers"]['text']}, 'id':  squad_fi[i]["id"]}
  p = {'prediction_text': predictions[i]["answer"], 'id': squad_fi[i]["id"], 'no_answer_probability': 0.0}
  gold.append(g)
  preds.append(p)

results = squad_metric.compute(predictions=preds, references=gold)
results